In [1]:
import keras as ks
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import re
from scipy.sparse import csr_matrix
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set(); sns.set_palette('husl')
mpl.rcParams['font.family'] = 'Arial Unicode MS'
from pprint import pprint

Using TensorFlow backend.


In [3]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [17]:
all = pd.concat([train, test], axis =0).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
all

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736


In [19]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

kesson_table(all)

,欠損数,%
Age,263,20.091673
Cabin,1014,77.463713
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
all["surname"] = all["Name"].apply(lambda x:x.split(",")[0])

苗字、性別、出港地をカテゴリ変数化

In [21]:
cols = ["surname", "Sex","Embarked"]
all_to_cat = pd.get_dummies(all, columns=cols)
print(all_to_cat.shape)

(1309, 890)


In [0]:
data_counts = all['surname'].value_counts()
surname_to_drop = data_counts[data_counts < 3].index.tolist()

to_drop_list = []
target_col = "surname"

for each_data in surname_to_drop:
    col_name = "{}_{}".format(target_col, each_data)
    to_drop_list.append(col_name)
    
all_to_cat.drop(columns=to_drop_list, inplace=True)

In [23]:
to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]

all_to_cat.drop(columns=to_drop, inplace=True)
print(all_to_cat.shape)

(1309, 116)


In [0]:
all_to_cat['Age'].fillna(all_to_cat['Age'].mode()[0], inplace=True)

In [25]:
all_to_cat

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,8.0500,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
5,24.0,8.4583,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6,54.0,51.8625,0,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
7,2.0,21.0750,1,3,3,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
8,27.0,11.1333,2,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
9,14.0,30.0708,0,2,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [27]:
train_to_cat = all_to_cat.iloc[0:891, :]
train_to_cat

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,35.0,8.0500,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
5,24.0,8.4583,0,3,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6,54.0,51.8625,0,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
7,2.0,21.0750,1,3,3,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
8,27.0,11.1333,2,3,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
9,14.0,30.0708,0,2,1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [28]:
all_to_cat["Pclass"] /= 3
all_to_cat["Age"] /= 80
all_to_cat["SibSp"] /= 8
all_to_cat["Parch"] /= 9
all_to_cat["Fare"] /= 512.3292

all_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.2750,0.014151,0.0,1.000000,0.125,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0.4750,0.139136,0.0,0.333333,0.125,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,0.3250,0.015469,0.0,1.000000,0.000,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0.4375,0.103644,0.0,0.333333,0.125,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,0.4375,0.015713,0.0,1.000000,0.000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 32
num_classes = 2
epochs = 20

In [0]:
from sklearn.model_selection import train_test_split

In [31]:
train_train = train_to_cat.drop(["Survived"], axis=1)
train_train.head()

,Age,Fare,Parch,Pclass,SibSp,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,surname_Graham,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.2750,0.014151,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0.4750,0.139136,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,0.3250,0.015469,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0.4375,0.103644,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,0.4375,0.015713,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [0]:
target = train_to_cat["Survived"].values
features = train_train.values

In [0]:
x_train, x_test, t_train, t_test = train_test_split(features, target, test_size =0.3, random_state = 0)

In [34]:
x_train.shape

(623, 115)

In [35]:
x_test.shape

(268, 115)

In [0]:
x_train = x_train.reshape(623, 115)
x_test = x_test.reshape(268, 115)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
t_train = ks.utils.to_categorical(t_train, num_classes)
t_test = ks.utils.to_categorical(t_test, num_classes)

In [38]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(115, )))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

W0703 10:39:14.668224 140503896160128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0703 10:39:14.709152 140503896160128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 10:39:14.717490 140503896160128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0703 10:39:14.740993 140503896160128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0703 10:39:14.750457 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                3712      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total params: 3,994
Trainable params: 3,994
Non-trainable params: 0
_________________________________________________________________


In [39]:
history = model.fit(x_train, t_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, t_test))

Train on 623 samples, validate on 268 samples
Epoch 1/20
623/623 [==============================] - 1s 1ms/step - loss: 0.6590 - acc: 0.6116 - val_loss: 0.6264 - val_acc: 0.6269
Epoch 2/20
623/623 [==============================] - 0s 57us/step - loss: 0.6329 - acc: 0.6116 - val_loss: 0.5979 - val_acc: 0.6306
Epoch 3/20
623/623 [==============================] - 0s 54us/step - loss: 0.6070 - acc: 0.6372 - val_loss: 0.5722 - val_acc: 0.6642
Epoch 4/20
623/623 [==============================] - 0s 53us/step - loss: 0.5895 - acc: 0.6838 - val_loss: 0.5520 - val_acc: 0.7015
Epoch 5/20
623/623 [==============================] - 0s 49us/step - loss: 0.5660 - acc: 0.7014 - val_loss: 0.5388 - val_acc: 0.7649
Epoch 6/20
623/623 [==============================] - 0s 59us/step - loss: 0.5645 - acc: 0.7063 - val_loss: 0.5290 - val_acc: 0.7910
Epoch 7/20
623/623 [==============================] - 0s 52us/step - loss: 0.5671 - acc: 0.7400 - val_loss: 0.5223 - val_acc: 0.7910
Epoch 8/20
623/623 [====

In [41]:
test_to_cat = all_to_cat.iloc[891:, :]
test_to_cat

,Age,Fare,Parch,Pclass,SibSp,Survived,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
891,0.43125,0.015282,0.000000,1.000000,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
892,0.58750,0.013663,0.000000,1.000000,0.125,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
893,0.77500,0.018909,0.000000,0.666667,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
894,0.33750,0.016908,0.000000,1.000000,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
895,0.27500,0.023984,0.111111,1.000000,0.125,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
896,0.17500,0.018006,0.000000,1.000000,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
897,0.37500,0.014891,0.000000,1.000000,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
898,0.32500,0.056604,0.111111,0.666667,0.125,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
899,0.22500,0.014110,0.000000,1.000000,0.000,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
900,0.26250,0.047138,0.000000,1.000000,0.250,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [44]:
test_test = test_to_cat.drop(["Survived"], axis=1)
test_test.head()

,Age,Fare,Parch,Pclass,SibSp,surname_Abbott,surname_Allison,surname_Andersson,surname_Asplund,surname_Baclini,surname_Becker,surname_Boulos,surname_Bourke,surname_Brown,surname_Cacic,surname_Caldwell,surname_Carlsson,surname_Carter,surname_Chapman,surname_Collyer,surname_Compton,surname_Cor,surname_Coutts,surname_Crosby,surname_Daly,surname_Danbom,surname_Davies,surname_Dean,surname_Dodge,surname_Douglas,surname_Drew,surname_Elias,surname_Flynn,surname_Ford,surname_Fortune,surname_Frauenthal,surname_Giles,surname_Goldsmith,surname_Goodwin,surname_Graham,...,surname_Olsen,surname_Olsson,surname_Oreskovic,surname_Palsson,surname_Panula,surname_Peacock,surname_Peter,surname_Phillips,surname_Quick,surname_Rice,surname_Richards,surname_Rosblom,surname_Ryerson,surname_Sage,surname_Samaan,surname_Sandstrom,surname_Skoog,surname_Smith,surname_Spedden,surname_Svensson,surname_Taussig,surname_Thayer,surname_Thomas,surname_Touma,surname_Van Impe,surname_Vander Planke,surname_Ware,surname_Warren,surname_Wells,surname_West,surname_White,surname_Wick,surname_Widener,surname_Williams,surname_van Billiard,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
891,0.43125,0.015282,0.000000,1.000000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
892,0.58750,0.013663,0.000000,1.000000,0.125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
893,0.77500,0.018909,0.000000,0.666667,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
894,0.33750,0.016908,0.000000,1.000000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
895,0.27500,0.023984,0.111111,1.000000,0.125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [0]:
test_features = test_test.values

In [48]:
predict_classes = model.predict_classes(test_features, batch_size=batch_size)

predict_classes

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [49]:
predict_classes.shape

(418,)

In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(predict_classes, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("Solution5_2.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("Solution5_2.csv")

75%だった。残念